# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory ‘data’: File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

My name is Erin Werner, I am originally from Southern California but I am currently living in New York City. I love to run, cook, and go to stand up comedy shows. 

This is my last class before I take the Capstone next term and this is the only class I am taking at the moment. 

Right now, I work as a Software Engineer, but my ultimate goal is to be a Data Analyst. I love the complex challenges that data science and machine learning present. So, I am hoping that W261 gives me real world skills that I will be able to apply in my next job.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

In machine learning, error can be decomposed into two components: the error due to squared bias, which is the amount by which the expected model prediction differs from the true value, over the training data ($E[h(x^*)] - f(x^*)$) and then the error due to variance, which is the amount by which the prediction, over one training set, differs from the expected predicted value, over all the training sets ($E[h(x^*)] - E[h(x^*)]^2$). So, the goal of the bias-variance tradeoff is to minimize both error types at the same time, while maintaining both flexibility and complexity in the machine learning model, so as not to over or under  fit the data.

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ 

The implementation of a string tokenizer can impact the final results of the machine learning model. This is because words can be tokenized in different ways and, thus, result in different data and results. Different tokenizers might count certain raw words as the same or as distinct tokens. This, then, affects both the total number of (unique) words as well as the individual word counts for those words. So, two different tokenizers can cause different calculations. For example, Tokenizer A removes special characters and numbers with " " (space) replacement and lowers the case of all the text. Then, Tokenizer B removes special characters and numbers with "" (no space) replacement and does not lower the case of the text. As a result, for the following example `tokenize("By-the-bye, what became of Alice's 12 hats?!")`, "Alice" and "alice" could be seen as two different strings depending on the tokenizer, which would result in different count calculations for "alice". The tokenizer that does not lowercase words (Tokenizer B) would calculate the likelihood of “alice” to be 1/9 compared to the tokenizer that did lowercase all the words (Tokenizer A), the likelihood of “alice” would be 2/8. 

> __b)__

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']

In [4]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile(r"([a-zA-Z]+)")          

In [5]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [6]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
#!curl -L "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt
!curl "https://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0  1182k      0 --:--:-- --:--:-- --:--:-- 1182k


In [7]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

﻿The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [8]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [9]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt
alice_test.txt.output
alice_test.txt.temp


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ 

Given that we now want to include the plural words in our word count calculations, I would choose to post-process my output file rather than discard the results and start the analysis over with a new tokenizer. Although the new tokenizer could provide some flexibility in how the words are stemmed/tokenized, that would require much more work compared to post-processing the file. After having run the wordcount, the plural version of the words have such small counts so starting over would not be efficient in this context. Post-processing would only require writing a few lines of code, using diffferent piping commands, compared to re-running the entire process from scratch after creating a new tokenizer.

In [10]:
# part a - DO YOUR WORK IN wordCount.py

In [11]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [12]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [13]:
!head data/alice_counts.txt

the	1839
project	88
gutenberg	98
ebook	13
of	638
alice	403
s	222
adventures	11
in	435
wonderland	7


__Check your results:__ How many times does the word "alice" appear in the book? 

In [14]:
# EXPECTED OUTPUT: 403
!grep alice data/alice_counts.txt

alice	403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [15]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [16]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting Practice
Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) short response:__ What is the Big O complexity of the fastest comparison based sorting algorithms? [*HINT: If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*]

* __b) short response:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*HINT: Julius Ceasar! (week 1 slides)*]

* __c) code:__ write a unix command to check how many records are in your word count file.

* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*HINT: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) code:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ 

The fastest comparison based sorting algorithm is Quicksort and its Big O complexity is O(n * log(n)).

> __b)__ 

The default sorting algorithm in MapReduce is Mergesort and its Big O complexity is O(n * log(n)). This algorithm was choosen because the data comes from the different nodes to a single point, so the best algorithm that can be used for that type of structure is the Mergesort.

In [17]:
# part c - write a unix command to check how many records are in your word count file
!wc -l < data/alice_counts.txt

3006


In [18]:
# part d - unix command to sort your word counts alphabetically 
!sort -o data/alice_counts_A-Z.txt data/alice_counts.txt

In [19]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [20]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -nr -k2,2 data/alice_counts.txt > data/alice_counts_sorted.txt 

In [21]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1837
and	946
to	811
a	695
of	637
it	610
she	553
i	546
you	487
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ Type your answer here!

In this case, we split the data up into four batches, but did not "reduce" or aggregate the count data back together after processing. That is why there are four separate instances of "alice" with different counts. We can see that the counts still add up to the correct value. So, we need to aggregate these batch-seperated values back together in pWordCount_v1.sh. This can be accomplished by piping the batch results to the aggregateCounts_v1.py function, which will then combine the values back together and output the final results file.

In [22]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [23]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [24]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	42


In [25]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [26]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [27]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


In [28]:
!man sort

SORT(1)                          User Commands                         SORT(1)

NAME
       sort - sort lines of text files

SYNOPSIS
       sort [OPTION]... [FILE]...
       sort [OPTION]... --files0-from=F

DESCRIPTION
       Write sorted concatenation of all FILE(s) to standard output.

       With no FILE, or when FILE is -, read standard input.

       Mandatory  arguments  to  long  options are mandatory for short options
       too.  Ordering options:

       -b, --ignore-leading-blanks
              ignore leading blanks

       -d, --dictionary-order
              consider only blanks and alphanumeric characters

       -f, --ignore-case
              fold lower case to upper case characters

       -g, --general-numeric-sort
              compare according to general numerical value

       -i, --ignore-nonprinting
              consider only printable characters

       -M, --month-sort
              compare (unknown) < 'JAN' < ... < 'DEC'

       -h, --human-numeric-sort
  

# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ 

The potential scalability problem with the provided implementation has to do with memory and storage. If there is a really large input corpus, the vocabulary would be too large to fit into memory, so the solution would be too big for one machine. This is because the solution uses a dictionary object that takes up space, where the data unsorted and stored with hashes. The Map-Reduce algorithm uses the mergesort algorithm, which can be optimized with sorted data. So, the supposedly "parallelized" word count would not work on a really large corpus because a large, unsorted vocabulary would take up too much memory for a single machine to process.

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ 

If we were to split the word counts in half after sorting them, we could run into some issues. An arbitrary split of the data could possibly lead to issues that still require postprocessing the data. For example, if we were to split the data in half, it is possible that we would be separating instances of the same word. This would then mess up the count algorithm and produce incorrect results unless we did another postprocessing step. Therefore, simply splitting the data in half is not the best idea for streaming the data to different machines.

> __e)__ 

In order to split up the data without requiring additional postprocessing, we could split up the data by the starting letter of the word. So, "a"'s could be grouped together, then "b"'s, and so on. In this case, all the same words would definitely be grouped together and there would be no opportunity to split the same instance of a word and throw off the overall count. Thus, we would not need additioanl postprocessing for any of these results with this approach.

In [29]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [30]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [31]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [32]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ 

Lin and Dyer consider the two features of an "ideal algorithm" (from a scability perspective) to be:

1.) In terms of data: that given twice the amount of data, the same algorithm should take at most twice as long to run, all else being equal. 

2.) In terms of resources: that given a cluster twice the size, the same algorithm should take no more than half as long to run.

So, essentially, an ideal algorithm would have a runtime that is linearly proportinal to the amount of data as well as the size of the cluster. Furthermore, an ideal algorithm would maintain these desirable scaling characteristics across a wide range of settings: on data ranging from gigabytes to petabytes, on clusters consisting of a few to a few thousand machines. Finally, the ideal algorithm would exhibit these desired behaviors without requiring any modifications whatsoever, not even tuning of parameters.

> __b)__ 

I can observe that when we use 1, 2, and 4 partitions, the "algorithm" takes a fair amount of time (seconds) to run. It is clear that the fewer the partitions, the longer the data takes to run. So, one partition will take the longest (4.29 seconds), compared to four partitions taking the shortest amount of time (1.24 seconds). Thus, as the time reduces by half for each incremental partition (1, 2, and then 4), this algorithm does meet Lin and Dyer's criteria for an ideal algorithm as the runtime is lineraly proportional to the size of the data being processed with each data partition.

> __c)__ 

I can observe that the overall runtimes have reduced greatly compared to the previous implementation. The times went from seconds to miliseconds to process the data. This matches my expection as this utilizes our scalable implementation, which requires less memory in order to access our data. However, although the run time is quicker in comparison, the run time decreases from 1 to 2 partitions, but then increases for an increased amount of partitions. This aspect did not necessarily match my expectations. Yet, this might be due to the limitation of a single machine, rather than partitioning the data to multiple different machines. On a single machine, separate partitions might have to wait for resources to be available in order to process the data before they can also be processed, thus more partitions would take more time. So, although this implementation is scalable, we can conclude that more computing resources are required in order to truly optimize this solution.

> __d)__ Type your (OPTIONAL) answer here!

__Run the following cells to create the mapper referenced in `part b`__

In [33]:
!mkdir demo

mkdir: cannot create directory ‘demo’: File exists


In [34]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Overwriting demo/mapper.py


In [35]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [36]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

4.29 s ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.26 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.24 s ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [39]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

266 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

261 ms ± 7.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

291 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

379 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

567 ms ± 6.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

948 ms ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ 

An application is embarrassingly parallel if it rarely, or never, has to communicate with other tasks. It can easily divide and conquer a task, such as word counting. This is because we can divide up the word count data for processesing and the aggregate our results back together for the final result. No step is dependent on another, so everything can be processed simultaneously or "in parallel". This term describes the implementation of a task as it refers to the way in which a task is processed.

> __b)__ Type your (OPTIONAL) answer here!

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform